# ME 8813 HW3
Name: Donaven Lobo

In [2]:
# Import libraries
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination, ApproxInference
import networkx as nx
import matplotlib.pyplot as plt

c:\Users\donav\OneDrive\Desktop\Gatech\MechEng\ME 8813\envs\envhw3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Create Bayesian Network for Juggling Model

juggle_model = BayesianNetwork(
    [
        ("Battery", "Juggle"),
        ("Road", "Juggle"),
        ("Juggle", "Olga"),
    ]
)

In [8]:
# Define the parameters using CPT

# Battery is true when the battery is charged
cpd_battery = TabularCPD(
    variable="Battery", 
    variable_card=2, 
    values=[[0.05], [0.95]]
)

# Road is true when the road is even
cpd_road = TabularCPD(
    variable="Road", 
    variable_card=2, 
    values=[[0.4], [0.6]]
)

# Juggle is true when the ball isn't dropped
cpd_juggle = TabularCPD(
    variable = 'Juggle',
    variable_card=2, 
    values=[[0.9, 0.5, 0.3, 0.01], [0.1, 0.5, 0.7, 0.99]],
    evidence=['Battery', 'Road'], 
    evidence_card=[2, 2]
)

# Olga is true when she reports the ball has been dropped
cpd_olga = TabularCPD(
    variable="Olga", 
    variable_card=2, 
    values=[[0.1, 0.9], [0.9, 0.1]],  # Adjusted values to ensure sum of each row is equal to 1
    evidence=['Juggle'],
    evidence_card=[2]
)

In [9]:
# Associating the parameters with the model structure
juggle_model.add_cpds(cpd_battery, cpd_road, cpd_juggle, cpd_olga)

In [10]:
# Check if the model is valid, throw an exception otherwise
juggle_model.nodes()

NodeView(('Battery', 'Juggle', 'Road', 'Olga'))

In [11]:
# Print out the Joint Probablity Distribution
print('Probability distribution, P(Battery)')
print(cpd_battery)
print()
print('Probability distribution, P(Road)')
print(cpd_road)
print()
print('Probability distribution, P(Juggle)')
print(cpd_juggle)
print()
print('Probability distribution, P(Olga)')
print(cpd_olga)
print()

Probability distribution, P(Battery)
+------------+------+
| Battery(0) | 0.05 |
+------------+------+
| Battery(1) | 0.95 |
+------------+------+

Probability distribution, P(Road)
+---------+-----+
| Road(0) | 0.4 |
+---------+-----+
| Road(1) | 0.6 |
+---------+-----+

Probability distribution, P(Juggle)
+-----------+------------+------------+------------+------------+
| Battery   | Battery(0) | Battery(0) | Battery(1) | Battery(1) |
+-----------+------------+------------+------------+------------+
| Road      | Road(0)    | Road(1)    | Road(0)    | Road(1)    |
+-----------+------------+------------+------------+------------+
| Juggle(0) | 0.9        | 0.5        | 0.3        | 0.01       |
+-----------+------------+------------+------------+------------+
| Juggle(1) | 0.1        | 0.5        | 0.7        | 0.99       |
+-----------+------------+------------+------------+------------+

Probability distribution, P(Olga)
+---------+-----------+-----------+
| Juggle  | Juggle(0) | Ju

In [12]:
# List out all the independencies in the model
juggle_model.get_independencies()


(Road ⟂ Battery)
(Road ⟂ Olga | Juggle)
(Road ⟂ Olga | Juggle, Battery)
(Olga ⟂ Road, Battery | Juggle)
(Olga ⟂ Battery | Juggle, Road)
(Olga ⟂ Road | Juggle, Battery)
(Battery ⟂ Road)
(Battery ⟂ Olga | Juggle)
(Battery ⟂ Olga | Juggle, Road)

In [13]:
# create to inference objects
infer_VE = VariableElimination(juggle_model) # Variable Elimination Inference
infer_approx = ApproxInference(juggle_model) # Approximate Inference

In [18]:
# Suppose that Olga reports that Jason has dropped the ball. What effect does this have on your belief that the battery is low? Use the Variable Elimination to perform an exact inference.
battery_low = infer_VE.query(variables=['Battery'], evidence={'Olga': 1})
# Print the potsterior probability
print("Using variable elimination, the probability that the battery is low given that Olga reports that Jason has dropped the ball is: \n", battery_low)

Using variable elimination, the probability that the battery is low given that Olga reports that Jason has dropped the ball is: 
 +------------+----------------+
| Battery    |   phi(Battery) |
+============+================+
| Battery(0) |         0.1413 |
+------------+----------------+
| Battery(1) |         0.8587 |
+------------+----------------+


In [19]:
battery_low = infer_approx.query(variables=['Battery'], evidence={'Olga': 1})
# Print the potsterior probability
print("Using approximate inference, the probability that the battery is low given that Olga reports that Jason has dropped the ball is: \n", battery_low)

  0%|          | 0/10000 [00:00<?, ?it/s]c:\Users\donav\OneDrive\Desktop\Gatech\MechEng\ME 8813\envs\envhw3\lib\site-packages\pgmpy\sampling\base.py:582: FutureWarning: Passing a DataFrame to DataFrame.from_records is deprecated. Use set_index and/or drop to modify the DataFrame instead.
  df = pd.DataFrame.from_records(samples)
c:\Users\donav\OneDrive\Desktop\Gatech\MechEng\ME 8813\envs\envhw3\lib\site-packages\pgmpy\sampling\base.py:582: FutureWarning: Passing a DataFrame to DataFrame.from_records is deprecated. Use set_index and/or drop to modify the DataFrame instead.
  df = pd.DataFrame.from_records(samples)
100%|██████████| 10000/10000 [00:00<00:00, 94034.68it/s]

Using approximate inference, the probability that the battery is low given that Olga reports that Jason has dropped the ball is: 
 +------------+----------------+
| Battery    |   phi(Battery) |
+============+================+
| Battery(0) |         0.1402 |
+------------+----------------+
| Battery(1) |         0.8598 |
+------------+----------------+
